In [29]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import sqrt, pi, arctan2, cos, sin
from scipy.ndimage import uniform_filter

original_frame1 = cv2.imread("data/tennis492.jpg")
original_frame2 = cv2.imread("data/tennis493.jpg")

# convert to gray-scale
frame1 = cv2.cvtColor(original_frame1,cv2.COLOR_BGR2GRAY)
frame2 = cv2.cvtColor(original_frame2,cv2.COLOR_BGR2GRAY)

frame1 = cv2.resize(frame1, (256,256))
frame2 = cv2.resize(frame2, (256,256))

# Gets the optical flow [<dx,dy>] from two frames
def getOpticalFlow(imPrev, imNew):
    flow = cv2.calcOpticalFlowFarneback(imPrev, imNew, flow=None, pyr_scale=.5, levels=3, winsize=9, iterations=1, poly_n=3, poly_sigma=1.1, flags=cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
    return flow

flow = getOpticalFlow(frame1, frame2)

In [30]:
# Compute the Histogram of Optical Flow (HoF) from the given optica flow

def hof(flow, orientations=9, pixels_per_cell=(10, 10),
        cells_per_block=(3, 3), visualise=False, normalise=False, motion_threshold=1.):
    flow = np.atleast_2d(flow)
    if flow.ndim < 3:
        raise ValueError("Requires dense flow in both directions")

    if normalise:
        flow = sqrt(flow)

    if flow.dtype.kind == 'u':
        # convert uint image to float
        # to avoid problems with subtracting unsigned numbers in np.diff()
        flow = flow.astype('float')

    gx = np.zeros(flow.shape[:2])
    gy = np.zeros(flow.shape[:2])
    # gx[:, :-1] = np.diff(flow[:,:,1], n=1, axis=1)
    # gy[:-1, :] = np.diff(flow[:,:,0], n=1, axis=0)

    gx = flow[:,:,1]
    gy = flow[:,:,0]
    
    magnitude = sqrt(gx**2 + gy**2)
    orientation = arctan2(gy, gx) * (180 / pi) % 180

    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    bx, by = cells_per_block

    n_cellsx = int(np.floor(sx // cx))  # number of cells in x
    n_cellsy = int(np.floor(sy // cy))  # number of cells in y

    # compute orientations integral images
    orientation_histogram = np.zeros((n_cellsy, n_cellsx, orientations))
    subsample = np.index_exp[cy / 2:cy * n_cellsy:cy, cx / 2:cx * n_cellsx:cx]
    for i in range(orientations-1):
        #create new integral image for this orientation
        # isolate orientations in this range

        temp_ori = np.where(orientation < 180 / orientations * (i + 1),
                            orientation, -1)
        temp_ori = np.where(orientation >= 180 / orientations * i,
                            temp_ori, -1)
        # select magnitudes for those orientations
        cond2 = (temp_ori > -1) * (magnitude > motion_threshold)
        temp_mag = np.where(cond2, magnitude, 0)

        temp_filt = uniform_filter(temp_mag, size=(cy, cx))
        orientation_histogram[:, :, i] = temp_filt[subsample]

    ''' Calculate the no-motion bin '''
    temp_mag = np.where(magnitude <= motion_threshold, magnitude, 0)

    temp_filt = uniform_filter(temp_mag, size=(cy, cx))
    orientation_histogram[:, :, -1] = temp_filt[subsample]

    # now for each cell, compute the histogram
    hof_image = None

    if visualise:
        from skimage import draw

        radius = min(cx, cy) // 2 - 1
        hof_image = np.zeros((sy, sx), dtype=float)
        for x in range(n_cellsx):
            for y in range(n_cellsy):
                for o in range(orientations-1):
                    centre = tuple([y * cy + cy // 2, x * cx + cx // 2])
                    dx = int(radius * cos(float(o) / orientations * np.pi))
                    dy = int(radius * sin(float(o) / orientations * np.pi))
                    rr, cc = draw.bresenham(centre[0] - dy, centre[1] - dx,
                                            centre[0] + dy, centre[1] + dx)
                    hof_image[rr, cc] += orientation_histogram[y, x, o]

    """
    The fourth stage computes normalisation, which takes local groups of
    cells and contrast normalises their overall responses before passing
    to next stage. Normalisation introduces better invariance to illumination,
    shadowing, and edge contrast. It is performed by accumulating a measure
    of local histogram "energy" over local groups of cells that we call
    "blocks". The result is used to normalise each cell in the block.
    Typically each individual cell is shared between several blocks, but
    its normalisations are block dependent and thus different. The cell
    thus appears several times in the final output vector with different
    normalisations. This may seem redundant but it improves the performance.
    We refer to the normalised block descriptors as Histogram of Oriented
    Gradient (hog) descriptors.
    """

    n_blocksx = (n_cellsx - bx) + 1
    n_blocksy = (n_cellsy - by) + 1
    normalised_blocks = np.zeros((n_blocksy, n_blocksx,
                                  by, bx, orientations))

    for x in range(n_blocksx):
        for y in range(n_blocksy):
            block = orientation_histogram[y:y+by, x:x+bx, :]
            eps = 1e-5
            normalised_blocks[y, x, :] = block / sqrt(block.sum()**2 + eps)

    """
    The final step collects the hof descriptors from all blocks of a dense
    overlapping grid of blocks covering the detection window into a combined
    feature vector for use in the window classifier.
    """

    if visualise:
        return normalised_blocks.ravel(), hof_image
    else:
        return normalised_blocks.ravel()

In [28]:
hof_result = hof(flow)
print hof_result
print hof_result.shape

[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,  -2.66055072e-19
  -1.96920303e-34   1.25529637e-03]
(42849,)


In [220]:
def hofom(flow, orientations=4, magnitudes=4, pixels_per_cell=(8,8)):
    gx = flow[:,:,1]
    gy = flow[:,:,0]
    
    magnitude = sqrt(gx**2 + gy**2)
    orientation = arctan2(gy, gx) * (180 / pi) % 180
    
    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    ncx = sx//cx
    ncy = sy//cy
    
    #(256,256) -> (25,25,4,4)
    for i in range(ncx)

hofom(flow)

10


In [230]:
sx = 256
cx = 10
ncx = sx//cy
print ncx
range(ncx)

25


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [216]:
# Compute the Histogram of Optical Flow (HoF) from the given optica flow

def hofom(flow, orientations=4, magnitudes=4, pixels_per_cell=(10, 10),
        cells_per_block=(3, 3), normalise=False, motion_threshold=1.):
    flow = np.atleast_2d(flow)
    if flow.ndim < 3:
        raise ValueError("Requires dense flow in both directions")

    if normalise:
        flow = sqrt(flow)

    if flow.dtype.kind == 'u':
        flow = flow.astype('float')

    gx = np.zeros(flow.shape[:2])
    gy = np.zeros(flow.shape[:2])

    gx = flow[:,:,1]
    gy = flow[:,:,0]
    
    magnitude = sqrt(gx**2 + gy**2)
    orientation = arctan2(gy, gx) * (180 / pi) % 180

    sy, sx = flow.shape[:2]
    cx, cy = pixels_per_cell
    bx, by = cells_per_block

    n_cellsx = int(np.floor(sx // cx))  # number of cells in x
    n_cellsy = int(np.floor(sy // cy))  # number of cells in y

    # compute orientations integral images
    orientation_histogram = np.zeros((n_cellsy, n_cellsx, orientations))
    subsample = np.index_exp[cy / 2:cy * n_cellsy:cy, cx / 2:cx * n_cellsx:cx]
    
    for i in range(orientations-1):
        temp_ori = np.where(orientation < 180 / orientations * (i + 1),
                            orientation, -1)
        temp_ori = np.where(orientation >= 180 / orientations * i,
                            temp_ori, -1)
        # select magnitudes for those orientations
        cond2 = (temp_ori > -1) * (magnitude > motion_threshold)
        temp_mag = np.where(cond2, magnitude, 0)

        temp_filt = uniform_filter(temp_mag, size=(cy, cx))
        orientation_histogram[:, :, i] = temp_filt[subsample]

    ''' Calculate the no-motion bin '''
    temp_mag = np.where(magnitude <= motion_threshold, magnitude, 0)

    temp_filt = uniform_filter(temp_mag, size=(cy, cx))
    orientation_histogram[:, :, -1] = temp_filt[subsample]

    # now for each cell, compute the histogram
    hof_image = None

#     n_blocksx = (n_cellsx - bx) + 1
#     n_blocksy = (n_cellsy - by) + 1
#     normalised_blocks = np.zeros((n_blocksy, n_blocksx,
#                                   by, bx, orientations))

#     for x in range(n_blocksx):
#         for y in range(n_blocksy):
#             block = orientation_histogram[y:y+by, x:x+bx, :]
#             eps = 1e-5
#             normalised_blocks[y, x, :] = block / sqrt(block.sum()**2 + eps)
            
    magnitude_ravel = magnitude.ravel()
    m = orientation_histogram[0,0,0]
    r = np.zeros((n_cellsx,n_cellsy))
    
    for i in range(n_cellsx):
        for j in range(n_cellsy):
            for mi in range(magnitudes):
                m = orientation_histogram[i, j, mi]
                percentile = stats.percentileofscore(magnitude_ravel, m)
                index = int(np.floor((percentile - 0)/100 * ((magnitudes-1) - 0) + 0))
                r[mi,index] += 1

    return orientation_histogram.ravel()

In [235]:
 hofom(flow)

10
